# Customer Segmentation

## Libraries

In [6]:
import pandas as pd
import numpy as np
import matplotlib as mpl

import matplotlib.pyplot as plt
import seaborn as sns
import datetime, nltk, warnings
import matplotlib.cm as cm
import itertools
from ydata_profiling import ProfileReport
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import preprocessing, model_selection, metrics, feature_selection
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn import neighbors, linear_model, svm, tree, ensemble
from sklearn.decomposition import PCA
from sklearn.ensemble import AdaBoostClassifier

from wordcloud import WordCloud, STOPWORDS
from IPython.display import display, HTML

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)

warnings.filterwarnings("ignore")

plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
%matplotlib inline

## Data Preparation

I load the data. Once done, I also give some basic informations on the content of the dataframe: the type of the various variables, the number of null values and their percentage with respect to the total number of entries:

In [7]:
df = pd.read_csv('data.csv', encoding='ISO-8859-1', dtype={'CustomerID': str,'InvoiceID': str})
print('Dataframe dimensions:', df.shape)

#---------------
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
#---------------

#---------------
#For NULL values
null_info = pd.DataFrame(df.dtypes).T.rename(index={0:'column type'})
null_info = pd.concat([null_info, pd.DataFrame(df.isnull().sum()).T.rename(index={0:'null values (nb)'})])
null_info = pd.concat([null_info, pd.DataFrame(df.isnull().sum()/df.shape[0]*100).T.rename(index={0:'null values (%)'})])
display(null_info)
#---------------

display(df.head())

Dataframe dimensions: (541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
column type,object,object,object,int64,datetime64[ns],float64,object,object
null values (nb),0,0,1454,0,0,0,135080,0
null values (%),0.0,0.0,0.268311,0.0,0.0,0.0,24.926694,0.0


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [37]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID             object
Country                object
dtype: object

While looking at the number of null values in the dataframe, it is interesting to note that  ∼
 25% of the entries are not assigned to a particular customer. With the data available, it is impossible to impute values for the user and these entries are thus useless for the current exercise. So I delete them from the dataframe:

In [8]:
df.dropna(axis = 0, subset = ['CustomerID'], inplace = True)
print('Dimensions', df.shape)

#---------------
null_info = pd.DataFrame(df.dtypes).T.rename(index={0:'column type'})
null_info = pd.concat([null_info, pd.DataFrame(df.isnull().sum()).T.rename(index={0:'null values (nb)'})])
null_info = pd.concat([null_info, pd.DataFrame(df.isnull().sum()/df.shape[0]*100).T.rename(index={0:'null values (%)'})])
display(null_info)
#---------------

display(df.head())

Dimensions (406829, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
column type,object,object,object,int64,datetime64[ns],float64,object,object
null values (nb),0,0,0,0,0,0,0,0
null values (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


OK, therefore, by removing these entries we end up with a dataframe filled at 100% for all variables! Finally, I check for duplicate entries and delete them:

In [9]:
print('Duplicate entries: {}'.format(df.duplicated().sum()))
df.drop_duplicates(inplace = True)

Duplicate entries: 5225


## Exploring the contect of Variables

This dataframe contains 8 variables that correspond to:

**InvoiceNo**: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.<br>
**StockCode**: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.<br>
**Description**: Product (item) name. Nominal.<br>
**Quantity**: The quantities of each product (item) per transaction. Numeric.<br>
**InvoiceDate**: Invice Date and time. Numeric, the day and time when each transaction was generated.<br>
**UnitPrice**: Unit price. Numeric, Product price per unit in sterling.<br>
**CustomerID**: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.<br>
**Country**: Country name. Nominal, the name of the country where each customer resides.<br>

### Countries
<br>Here, I quickly look at the countries from which orders were made:

In [10]:
t1 = df[['CustomerID', 'InvoiceNo', 'Country']].groupby(['CustomerID','InvoiceNo','Country']).count()
t1 = t1.reset_index(drop = False)
countries = t1['Country'].value_counts()
print('No. of countries in the dataframe: {}'.format(len(countries)))

#---------------
#t2 = df.Country.unique()
#len(t2)
#---------------

No. of countries in the dataframe: 37


and show the result on a choropleth map:

In [11]:
data = dict(
            type = 'choropleth',
            locations = countries.index,
            locationmode = 'country names',
            text = countries.index,
            colorbar = {'title' : 'Order No. '},
            z = countries,
            colorscale=[[0, 'rgb(224,255,255)'], [0.01, 'rgb(166,206,227)'], [0.02, 'rgb(31,120,180)'],
                        [0.03, 'rgb(178,223,138)'], [0.05, 'rgb(51,160,44)'], [0.10, 'rgb(251,154,153)'], 
                        [0.20, 'rgb(255,255,0)'], [1, 'rgb(227,26,28)']],
            reversescale = False
        )

layout=dict(
            title = 'Numberof orders per country',
            geo = dict(showframe = True, projection = {'type':'mercator'})    
        )

choromap = go.Figure(data = [data], layout = layout)
iplot(choromap, validate = False)

We see that the dataset is largely dominated by orders made from the UK.

### Customers and Products


The dataframe contains  ∼400,000 entries. What are the number of users and products in these entries ?

In [20]:
cust_pros = pd.DataFrame([{
                            'products' : len(df['StockCode'].value_counts()),
                            'transactions' : len(df['InvoiceNo'].value_counts()),
                            'customers' : len(df['CustomerID'].value_counts())
            }],
            columns = ['products','transactions','customers'], index = ['quantity']
            )
cust_pros


,products,transactions,customers
quantity,3684,22190,4372


It can be seen that the data concern 4372 users and that they bought 3684 different products. The total number of transactions carried out is of the order of  ∼
 22'000.

Now I will determine the number of products purchased in every transaction:

In [35]:
temp = df.groupby(by=['CustomerID','InvoiceNo'], as_index=False)['InvoiceDate'].count()


products_per_basket = temp.rename(columns = {'InvoiceDate': 'Number of Products'}) 
products_per_basket[:10].sort_values('CustomerID')

,CustomerID,InvoiceNo,Number of Products
0,12346,541431,1
1,12346,C541433,1
2,12347,537626,31
3,12347,542237,29
4,12347,549222,24
5,12347,556201,18
6,12347,562032,22
7,12347,573511,47
8,12347,581180,11
9,12348,539318,17


The first lines of this list shows several things worthy of interest:

* the existence of entries with the prefix C for the InvoiceNo variable: this indicates transactions that have been canceled<br>
* the existence of users who only came once and only purchased one product (e.g. nº12346)<br>
* the existence of frequent users that buy a large number of items at each order

#### Cancelling Orders

Count the numbers of transactions corresponding to canceled orders

In [46]:
products_per_basket['orders canceled'] = products_per_basket['InvoiceNo'].apply(lambda x:int ('C' in x))
display(products_per_basket[:5])

#---------------
cansum = products_per_basket['orders canceled'].sum()
totals = products_per_basket.shape[0]
print('Number of orders canceled: {}/{} ({}%)'.format(cansum, totals, round(cansum/totals*100,2)))

,CustomerID,InvoiceNo,Number of Products,orders canceled
0,12346,541431,1,0
1,12346,C541433,1,1
2,12347,537626,31,0
3,12347,542237,29,0
4,12347,549222,24,0


Number of orders canceled: 3654/22190 (16.47%)


We note that the number of cancellations is quite large ( ∼
 16% of the total number of transactions). Now, let's look at the first lines of the dataframe:

In [47]:
display(df.sort_values('CustomerID')[:5])

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346,United Kingdom
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346,United Kingdom
286623,562032,22375,AIRLINE BAG VINTAGE JET SET BROWN,4,2011-08-02 08:48:00,4.25,12347,Iceland
72260,542237,84991,60 TEATIME FAIRY CAKE CASES,24,2011-01-26 14:30:00,0.55,12347,Iceland
14943,537626,22772,PINK DRAWER KNOB ACRYLIC EDWARDIAN,12,2010-12-07 14:57:00,1.25,12347,Iceland


On these few lines, we see that when an order is canceled, we have another transactions in the dataframe, mostly identical except for the Quantity and InvoiceDate variables. I decide to check if this is true for all the entries. To do this, I decide to locate the entries that indicate a negative quantity and check if there is systematically an order indicating the same quantity (but positive), with the same description (CustomerID, Description and UnitPrice):

In [57]:
dff = df[ df['Quantity'] < 0]  [['CustomerID','Quantity','StockCode','Description','UnitPrice']]

for index, col in dff.iterrows():
    if df[ (df['CustomerID'] == col[0]) & (df['Quantity'] == -col[1]) & (df['Description'] == col[2])].shape[0] == 0:
        print(dff.loc[index])
        print('---')
        print(15 * '-' + '>' + ' Hypothesis is not true')
        break

CustomerID        14527
Quantity             -1
StockCode             D
Description    Discount
UnitPrice          27.5
Name: 141, dtype: object
---
---------------> Hypothesis is not true


We see that the initial hypothesis is not fulfilled because of the existence of a 'Discount' entry. I check again the hypothesis but this time discarding the 'Discount' entries:

In [58]:
dff = df[ (df['Quantity'] < 0) & (df['Description'] != 'Discount')]  [['CustomerID','Quantity','StockCode','Description','UnitPrice']]

for index, col in dff.iterrows():
    if df[ (df['CustomerID'] == col[0]) & (df['Quantity'] == -col[1]) & (df['Description'] == col[2])].shape[0] == 0:
        print(dff.loc[index])
        print('---')
        print(15 * '-' + '>' + ' Hypothesis is not true')
        break

CustomerID                               15311
Quantity                                    -1
StockCode                               35004C
Description    SET OF 3 COLOURED  FLYING DUCKS
UnitPrice                                 4.65
Name: 154, dtype: object
---
---------------> Hypothesis is not true


In [62]:
cleandf = df.copy(deep = True)
cleandf['QuantityCanceled'] = 0

etr = [] ; de = []

for index, col in df.iterrows():
    if(col['Quantity'] > 0) or col['Description'] == 'Discount': continue
    dft = df[(df['CustomerID'] == col['CustomerID']) &
             (df['StockCode']  == col['StockCode'])  &
             (df['InvoiceDate'] < col['InvoiceDate']) &
             (df['Quantity']   > 0)].copy()
    
    # No CounterPart Found
    if (dft.shape[0] == 0) : 
            de.append(index)

    # With CounterPart
    elif (dft.shape[0] == 1):
        dft.sort_index(axis = 0, ascending = False, inplace = True)
        for ind,val in dft.iterrows():
             if val['Quantity'] < -col['Quantity'] : continue
             cleandf.loc[ind, 'QuantityCanceled'] = -col['Quantity']
             etr.append(index)
             break

In [63]:
print("entry_to_remove: {}".format(len(etr)))
print("doubtfull_entry: {}".format(len(de)))

entry_to_remove: 4750
doubtfull_entry: 1226
